# Airbnb NYC Rental Prices Analysis

**Author:** David Graham

**Dataset:** Airbnb NYC 2019 Listings

## Project Description

This project performs an exploratory data analysis on the Airbnb NYC 2019 dataset to uncover patterns and relationships between listing features and rental prices. Through data visualization and statistical analysis, we aim to identify key factors that influence pricing and discover meaningful insights about the short-term rental market in New York City.

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Load Dataset

In [ ]:
df = pd.read_csv("AB_NYC_2019.csv")
df.head()